## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
dropped_df = application_df.drop(columns=['EIN', 'NAME'])

In [6]:
# Determine the number of unique values in each column.
dropped_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning
apcount = dropped_df["APPLICATION_TYPE"].value_counts()
apcount

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = list(apcount[apcount < 100])

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
classcount = dropped_df["CLASSIFICATION"].value_counts()

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classcount.loc[classcount > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classcount[classcount < 700])
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
new_df = pd.get_dummies(dropped_df)

In [13]:
# Split our preprocessed data into our features and target arrays
y = new_df["IS_SUCCESSFUL"].values
X = new_df.drop("IS_SUCCESSFUL", axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state= 42)


In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input_feat = len(X_train[0])
hidden_layer1 = 7
hidden_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer1, input_dim=num_input_feat, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 7)                 819       
                                                                 
 dense_1 (Dense)             (None, 5)                 40        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 865
Trainable params: 865
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
804/804 [==============================] - 1s 871us/step - loss: 0.5393 - accuracy: 0.7352
Epoch 2/200
804/804 [==============================] - 1s 927us/step - loss: 0.5391 - accuracy: 0.7356
Epoch 3/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5394 - accuracy: 0.7354
Epoch 4/200
804/804 [==============================] - 1s 843us/step - loss: 0.5391 - accuracy: 0.7357
Epoch 5/200
804/804 [==============================] - 1s 853us/step - loss: 0.5392 - accuracy: 0.7345
Epoch 6/200
804/804 [==============================] - 1s 851us/step - loss: 0.5389 - accuracy: 0.7351
Epoch 7/200
804/804 [==============================] - 1s 865us/step - loss: 0.5390 - accuracy: 0.7363
Epoch 8/200
804/804 [==============================] - 1s 881us/step - loss: 0.5388 - accuracy: 0.7358
Epoch 9/200
804/804 [==============================] - 1s 879us/step - loss: 0.5391 - accuracy: 0.7371
Epoch 10/200
804/804 [==============================] - 1s 883us/step - los

804/804 [==============================] - 1s 866us/step - loss: 0.5377 - accuracy: 0.7363
Epoch 80/200
804/804 [==============================] - 1s 829us/step - loss: 0.5380 - accuracy: 0.7371
Epoch 81/200
804/804 [==============================] - 1s 844us/step - loss: 0.5379 - accuracy: 0.7365
Epoch 82/200
804/804 [==============================] - 1s 859us/step - loss: 0.5378 - accuracy: 0.7362
Epoch 83/200
804/804 [==============================] - 1s 840us/step - loss: 0.5380 - accuracy: 0.7367
Epoch 84/200
804/804 [==============================] - 1s 854us/step - loss: 0.5379 - accuracy: 0.7366
Epoch 85/200
804/804 [==============================] - 1s 836us/step - loss: 0.5377 - accuracy: 0.7367
Epoch 86/200
804/804 [==============================] - 1s 860us/step - loss: 0.5378 - accuracy: 0.7377
Epoch 87/200
804/804 [==============================] - 1s 837us/step - loss: 0.5378 - accuracy: 0.7374
Epoch 88/200
804/804 [==============================] - 1s 834us/step - loss:

804/804 [==============================] - 1s 728us/step - loss: 0.5372 - accuracy: 0.7377
Epoch 158/200
804/804 [==============================] - 1s 768us/step - loss: 0.5372 - accuracy: 0.7371
Epoch 159/200
804/804 [==============================] - 1s 735us/step - loss: 0.5373 - accuracy: 0.7365
Epoch 160/200
804/804 [==============================] - 1s 763us/step - loss: 0.5374 - accuracy: 0.7378
Epoch 161/200
804/804 [==============================] - 1s 707us/step - loss: 0.5374 - accuracy: 0.7365
Epoch 162/200
804/804 [==============================] - 1s 702us/step - loss: 0.5374 - accuracy: 0.7367
Epoch 163/200
804/804 [==============================] - 1s 724us/step - loss: 0.5372 - accuracy: 0.7374
Epoch 164/200
804/804 [==============================] - 1s 748us/step - loss: 0.5374 - accuracy: 0.7374
Epoch 165/200
804/804 [==============================] - 1s 718us/step - loss: 0.5372 - accuracy: 0.7380
Epoch 166/200
804/804 [==============================] - 1s 808us/ste

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5536 - accuracy: 0.7287 - 286ms/epoch - 1ms/step
Loss: 0.5536479949951172, Accuracy: 0.7287463545799255


In [16]:
# Export our model to HDF5 file
#nn.save("AlphabetSoupCharity_Optimization.h5")